## Exemple d'interrogation du point d'accès SPARQL de la Bibliothèque nationale de France (BNF)

Dans cet exemple, après avoir interrogé le point d'accès SPARQL de la BNF, on extrait des données reçues l'année de naissance de la population et on propose différentes distributions permettant de mettre en évitence l'évolution des effectifs des auteurs d'ouvrages conservés à la BNF ayant été actifs dans les disciplines des mathématiques et de l'astronomie.

On créer également un document CSV exportable et analysable avec d'autres outils (un tableur par ex.).


### Importer les librairies à utiliser

In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from collections import Counter
from operator import itemgetter

### Préparation et exécution de la requête SPARQL

In [2]:
query = """
SELECT ?prop ?propLabel (COUNT(*) AS ?freq) WHERE {
  # hint:Query hint:optimizer "None" .
  ?s ?p wd:Q188094.
  ?prop wikibase:directClaim ?p.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?prop ?propLabel
ORDER BY DESC (?freq)
"""

In [6]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql") ##, returnFormat=RDFXML)

In [7]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [8]:
rc = sparql.queryAndConvert()
type(rc)

dict

### Analyse du résultat

In [9]:
# Nombre de lignes du résultat
len(rc['results']['bindings'])

17

In [10]:
# Inspecter les trois premières lignes
i = 0
for l in rc['results']['bindings']:
    if i < 3:
        print(l)
        i += 1

{'prop': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P106'}, 'propLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'occupation'}, 'freq': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '34755'}}
{'prop': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P971'}, 'propLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'category combines topics'}, 'freq': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '112'}}
{'prop': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P101'}, 'propLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'field of work'}, 'freq': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '69'}}


In [ ]:
# Extraire les valeurs des cinq premières lignes
i = 0
for l in rc['results']['bindings']:
    if i < 5:
        try: 
            name = l['name']['value']
            print([l['s']['value'],name,l['annee']['value'],l['bio']['value']])
        except :
            try: 
                name = l['prefLabel']['value']
                print([l['s']['value'],name,l['annee']['value'],l['bio']['value']])
            except :
                print([l['s']['value'],'',l['annee']['value'],l['bio']['value']])

        i += 1

### Sauvegarder le résultat sous forme de document CSV
Documentation: https://docs.python.org/3/library/csv.html

In [ ]:
with open('sparql_bnf_1.csv', 'w', newline='') as csvfile:
    document = csv.writer(csvfile, delimiter='|',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for l in rc['results']['bindings']:
        try: 
            name = l['name']['value']
            document.writerow([l['s']['value'],name,l['annee']['value'],l['bio']['value']])
        except :
            try: 
                name = l['prefLabel']['value']
                document.writerow([l['s']['value'],name,l['annee']['value'],l['bio']['value']])
            except :
                document.writerow([l['s']['value'],'',l['annee']['value'],l['bio']['value']])


### Inspecter les années de naissance et compter le nombre de naissances par années

In [ ]:
# Explore
listeAnnees = []
for l in rc['results']['bindings']:
    listeAnnees.append(l['annee']['value'])

In [ ]:
listeAnnees[51:58]

In [ ]:
min(listeAnnees), max(listeAnnees)

In [ ]:
listeAnnees.sort()

In [ ]:
# Compter les naissances par année
counterListeAnnees = Counter(listeAnnees)
groupedListeAnnees = list(zip(counterListeAnnees.keys(), counterListeAnnees.values()))

In [ ]:
groupedListeAnnees[:5]

In [ ]:
# Transformer les années de chaines de caractères à chiffres
groupedListeAnnees = [[int(i[0]), i[1]] for i in groupedListeAnnees]
groupedListeAnnees[:5]

In [ ]:
### Créer la liste des périodes de 20 ans, i.e. la liste de la première année de chaque période
d_list = []
period = 20
d_start = 1001
i = 0
d_list = []
d_list.append(d_start)

while i < 50:
    d_start = d_start + period
    d_list.append(d_start)
    i += 1

In [ ]:
print(d_list[:2])
min(d_list), max(d_list)

### Distribution des naissances par année et par période de 20 ans

In [ ]:
# Distribution par année

objects = [l[0] for l in groupedListeAnnees]
eff = [l[1] for l in groupedListeAnnees]

p1 = plt.bar(objects, eff)

plt.xticks(d_list)
plt.ylabel('Effectif')
plt.title('Naissances par année')
plt.gcf().set_size_inches(16, 8)
#plt.gca().axes.get_xaxis().set_visible(False)
# supported formats: eps, pdf, pgf, png, ps, raw, rgba, svg, svgz
#plt.savefig('exported_pictures/effectifs_naissances_par_annee.png', dpi=100, bbox_inches = "tight") #pdf, svg
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Regroupement par période de 20 ans
i_list = []
for i in d_list:
    i_n = 0
    for l in groupedListeAnnees:
        if l[0] >= i and l[0] < (i + 20):
            i_n = i_n + l[1]
    i_list.append([i, i_n])        

In [ ]:
i_list[30:35]

In [ ]:
# Distribution par période de 20 ans

objects = [str(l[0]) for l in i_list]
eff = [l[1] for l in i_list]

p1 = plt.bar(objects, eff)

plt.ylabel('Effectif')
plt.title('Naissances par période de vingts ans')
plt.gcf().set_size_inches(16, 8)
# supported formats: eps, pdf, pgf, png, ps, raw, rgba, svg, svgz
# plt.savefig('exported_pictures/effectifs_annees_reference_par_periode.png', dpi=100, bbox_inches = "tight") #pdf, svg
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Distribution par période de 20 ans, seulement après 1451

objects = [str(l[0]) for l in i_list if l[0] >= 1451]
eff = [l[1] for l in i_list if l[0] >= 1451]

p1 = plt.bar(objects, eff)

plt.ylabel('Effectif')
plt.title('Naissances par période de vingts ans')
plt.gcf().set_size_inches(16, 8)
# supported formats: eps, pdf, pgf, png, ps, raw, rgba, svg, svgz
# plt.savefig('exported_pictures/effectifs_annees_reference_par_periode.png', dpi=100, bbox_inches = "tight") #pdf, svg
plt.xticks(rotation=45)
plt.show()


Regroupement par 50 ans

In [ ]:
### Créer la liste des périodes de 50 ans, i.e. la liste de la première année de chaque période
d_list = []
period = 50
d_start = 1001
i = 0
d_list = []
d_list.append(d_start)

while i < 20:
    d_start = d_start + period
    d_list.append(d_start)
    i += 1

In [ ]:
print(d_list[:2])
min(d_list), max(d_list)

In [ ]:
# Regroupement par période de 50 ans
i_list = []
for i in d_list:
    i_n = 0
    for l in groupedListeAnnees:
        if l[0] >= i and l[0] < (i + 20):
            i_n = i_n + l[1]
    i_list.append([i, i_n])        

In [ ]:
i_list[10:15]

In [ ]:
# Distribution par période de 50 ans

objects = [str(l[0]) for l in i_list]
eff = [l[1] for l in i_list]

p1 = plt.bar(objects, eff)

plt.ylabel('Effectif')
plt.title('Naissances par période de cinquant ans')
plt.gcf().set_size_inches(16, 8)
# supported formats: eps, pdf, pgf, png, ps, raw, rgba, svg, svgz
# plt.savefig('exported_pictures/effectifs_annees_reference_par_periode.png', dpi=100, bbox_inches = "tight") #pdf, svg
plt.xticks(rotation=45)
plt.show()
